In [1]:
#age in code ro nafahmidin khodetoon ye code joda ba estefade az StreamData_preprocessed.csv benevisin. yani
#ye code benevisin ke StreamData_preprocessed.csv ro load kone va satr hasho yeki yeki pass bede be codetoon.
#input: 7-column-rows
#output: just save it to cassandra. and write an app to get a hashtag as input and show the articles about that hashtag in python (show the output in console).
import json,datetime
from kafka import KafkaConsumer
import pandas as pd
import datetime

topicToReceive = 'cassandra'
kafkaPort = 19092
cassandraPort = 9042

In [2]:
from cassandra.cluster import Cluster
from cassandra.query import SimpleStatement
import pandas as pd

# Connect to Cassandra cluster
cluster = Cluster(['127.0.0.1'], port=cassandraPort)
session = cluster.connect()
print('Connected to cassanda.')

Connected to cassanda.


In [11]:
keyspace = 'telegram_news'
session.execute("CREATE KEYSPACE IF NOT EXISTS telegram_news WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1}")
session.set_keyspace(keyspace)

In [13]:
create_table_query = """
    CREATE TABLE IF NOT EXISTS telegram_news.hashtag (
        message text,
        hashtag text,
        PRIMARY KEY (message, hashtag)
    )
"""
session.execute(create_table_query)

In [14]:
create_table_query = """
    CREATE TABLE IF NOT EXISTS telegram_news.date (
        date timestamp,
        year int,
        month int,
        day int,
        hour int,
        minute int,
        message text,
        PRIMARY KEY (date, year, month, day, hour)
    )
"""
session.execute(create_table_query)

In [3]:
def current_time():
    current_datetime = datetime.datetime.now()
    return current_datetime.strftime("[%Y-%m-%d_%H:%M:%S]")

consumer = KafkaConsumer(topicToReceive, bootstrap_servers=[f'127.0.0.1:{kafkaPort}'])

In [4]:
import threading
import queue
import time
MsgQueue = queue.Queue()
def QueueProcessor():
    global MsgQueue
    while True:
        time.sleep(0.1)
        if not MsgQueue.empty():
            row = MsgQueue.get()
            # 1
            date_str = row['date']
            date = datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S%z')
            year = date.year
            month = date.month
            day = date.day
            hour = date.hour
            minute = date.minute
            message = row['message']
            insert_query = "INSERT INTO telegram_news.date (date, year, month, day, hour, minute, message) VALUES (%s, %s, %s, %s, %s, %s, %s)"
            session.execute(insert_query, (date, year, month, day, hour, minute, message))
            # 2
            hashtags = row['hashtags']
            hashtags = ast.literal_eval(hashtags)
            if len(hashtags) == 0:
                continue
            # If there is a single item, add it as a record in the table
            if len(hashtags) == 1:
                hashtag = hashtags[0]
                insert_query = "INSERT INTO telegram_news.hashtag (message, hashtags) VALUES (%s, %s)"
                session.execute(insert_query, (message, hashtag))
            else:
                # If there are multiple items, add each one separately as separate records
                for hashtag in hashtags:
                    insert_query = "INSERT INTO telegram_news.hashtag (message, hashtags) VALUES (%s, %s)"
                    session.execute(insert_query, (message, hashtag))

MsgQueueThread = threading.Thread(target=MsgQueue)
MsgQueueThread.start()
print(f"{current_time()}: MsgQueue is running.")

[2023-07-17_18:35:02]: MsgQueue is running.


Exception in thread Thread-3:
Traceback (most recent call last):
  File "c:\Users\AhmaDGoly\AppData\Local\Programs\Python\Python39\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "c:\Users\AhmaDGoly\AppData\Local\Programs\Python\Python39\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
TypeError: 'Queue' object is not callable


In [5]:
x = 0
for message in consumer:
    x = x + 1
    print(f"{current_time()}: {x}: Received a news from Preprocessing_Thread. putting it to cassandra...")
    json_row = message.value.decode('utf-8')
    row = json.loads(json_row)
    #receivedRowDF = pd.DataFrame([row])

    #az inja be bad kodetoon ro be ezaye har satre jadidi ke omad piade konin.
    MsgQueue.put(row)

[2023-07-17_18:36:40]: 1: Received a news from Preprocessing_Thread. putting it to cassandra...
[2023-07-17_18:37:06]: 2: Received a news from Preprocessing_Thread. putting it to cassandra...
[2023-07-17_18:37:33]: 3: Received a news from Preprocessing_Thread. putting it to cassandra...
[2023-07-17_18:38:02]: 4: Received a news from Preprocessing_Thread. putting it to cassandra...
[2023-07-17_18:38:34]: 5: Received a news from Preprocessing_Thread. putting it to cassandra...
[2023-07-17_18:39:02]: 6: Received a news from Preprocessing_Thread. putting it to cassandra...
[2023-07-17_18:39:33]: 7: Received a news from Preprocessing_Thread. putting it to cassandra...
[2023-07-17_18:40:05]: 8: Received a news from Preprocessing_Thread. putting it to cassandra...
[2023-07-17_18:40:35]: 9: Received a news from Preprocessing_Thread. putting it to cassandra...
[2023-07-17_18:41:05]: 10: Received a news from Preprocessing_Thread. putting it to cassandra...
[2023-07-17_18:41:32]: 11: Received a n

KeyboardInterrupt: 